*Training Poker Using RLLib*

In [2]:
import sys
from poker_env import PokerEnv
from agents.random_policy import RandomActions
from agents.heuristic_policy import HeuristicPolicy
from ray.rllib.algorithms.ppo import PPOConfig
from gym import spaces
import mpu
import numpy as np
import ray
from ray.rllib.models import MODEL_DEFAULTS
from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env
from IPython.display import clear_output

/usr/lib64/python3.7/site-packages/simplejson/compat.py:23: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload as reload_module
/home/ec2-user/RL_Poker_Env/poker_env.py:10: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In Rllib, a policy function needs to be passed to map agent IDs to the policy to use. We also create and register the learning environment.

In [3]:
def select_policy(agent_id, episode, **kwargs):
    if agent_id == 0:
        return "learned"
    elif agent_id == 1:
        return "Heuristic_10"
    elif agent_id == 2:
        return "Heuristic_100"
    elif agent_id == 3:
        return "random"
    return "Heuristic_1000"

def env_creator(config):
    env = PokerEnv(select_policy, config)
    return env

register_env("poker", lambda config: env_creator(config))

The config describes all aspects of the training. A full list of the parameters is found here: https://github.com/ray-project/ray/blob/master/rllib/algorithms/algorithm_config.py 

In this example, we have used a default config that runs the PPO algorithm. Other heuristic agents have been defined that will play the game with the single learning agent. 

In [4]:
heuristic_observation_space = spaces.Dict({
            "hand": spaces.Box(0, 1, shape=(24, )),
            "community": spaces.Box(0, 1, shape=(24, ))
        })
action_space = spaces.Discrete(3)

#Defines the learning models architecture. 
model = MODEL_DEFAULTS.update({'fcnet_hiddens': [512, 512], 'fcnet_activation': 'relu'})
#model={"fcnet_hiddens": [256, 256], "fcnet_activation": "tanh",}

config = (
    PPOConfig()
    #Each rollout worker uses a single cpu
    .rollouts(num_rollout_workers=1, num_envs_per_worker=1)\
    .training(train_batch_size=4000, gamma=0.99, model=model, lr=0.0004)\
    .environment(disable_env_checking=True)\
    .multi_agent(
        policies={
            #These policies thave pre-definded polices that dont learn.
            "random": PolicySpec(policy_class=RandomActions),
            "Heuristic_10": (HeuristicPolicy, heuristic_observation_space, action_space, {'difficulty': 0}),
            "Heuristic_100": (HeuristicPolicy, heuristic_observation_space, action_space, {'difficulty': 1}),
            "Heuristic_1000": (HeuristicPolicy, heuristic_observation_space, action_space, {'difficulty': 2}),
            #Passing nothing causes this agent to deafult to using a PPO policy
            "learned": PolicySpec(
                config={}
            ),
        },
        policy_mapping_fn=select_policy,
        policies_to_train=['learned'],
    )\
    .resources(num_gpus=0)\
    .framework('torch')
)
trainer = config.build(env="poker")

/home/ec2-user/.local/lib/python3.7/site-packages/ray/_private/ray_option_utils.py:273: DeprecationWarning: Setting 'object_store_memory' for actors is deprecated since it doesn't actually reserve the required object store memory. Use object spilling that's enabled by default (https://docs.ray.io/en/releases-2.0.0/ray-core/objects/object-spilling.html) instead to bypass the object store memory size limitation.
  stacklevel=1,
2023-03-28 07:53:51,510	INFO worker.py:1518 -- Started a local Ray instance.
(RolloutWorker pid=16039) /home/ec2-user/RL_Poker_Env/poker_env.py:10: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
(RolloutWorker pid=16039)   from collections import Iterable
2023-03-28 07:53:59,678	INFO trainable.py:164 -- Trainable.setup took 11.748 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.

In [4]:
#Start up tensorboard
#!tensorboard --logdir=~/ray_results --host 0.0.0.0

Training loop, each run will rollout x timesteps (where x is train_batch_size). An weight update is then applied using the rollout data. 

In [6]:
for i in range(1000):
    trainer.train()
    print(i)
    clear_output(wait=True)

ValueError: Could not find key 'grad_gnorm' in some `input_trees`. Please ensure the structure of all `input_trees` are compatible with `shallow_tree`. The last valid path yielded was ('learner_stats', 'entropy_coeff').

In [7]:
#Saves a checkpoint of the trainer.
trainer.save("checkpoint/ppo_poker")

'checkpoint/ppo_poker/checkpoint_000203'